In [25]:
import lxml.etree as ET


def extract_mode_mapping(cii_path: str) -> dict:
    tree = ET.parse(cii_path)
    root = tree.getroot()
    ns = {"xsl": "http://www.w3.org/1999/XSL/Transform"}
    mapping = {}

    templates = root.xpath("//xsl:template", namespaces=ns)
    for tmpl in templates:
        mode = tmpl.get("mode")
        if not mode:
            continue
        children = list(tmpl)
        if children:
            child_tag = ET.QName(children[0]).localname
            mapping[mode] = child_tag
    return mapping


def extract_id_mapping(xml_path: str) -> dict:
    tree = ET.parse(xml_path)
    root = tree.getroot()

    result = {}
    for entry in root.findall("entry"):
        item_id = entry.get("id")
        item_key = entry.get("key")
        if item_id and item_key:
            # Remove the 'xr:' prefix
            item_key = item_key.replace("xr:", "")
            result[item_id] = item_key

    return result


def extract_id_to_value_mapping(xml_path: str) -> dict:
    tree = ET.parse(xml_path)
    root = tree.getroot()

    result = {}
    for entry in root.findall("entry"):
        item_id = entry.get("id")
        item_value = entry.text.strip() if entry.text else None
        if item_id and item_value:
            result[item_id] = item_value

    return result

In [34]:
import pandas as pd


def combine_and_write_to_csv(xsl_path: str, xml_path: str, output_path: str):
    # Generate dictionaries using the provided functions
    dict1 = extract_mode_mapping(xsl_path)
    dict2 = extract_id_mapping(xml_path)
    dict3 = extract_id_to_value_mapping(xml_path)

    # Combine the dictionaries into a DataFrame
    df = pd.DataFrame(
        {
            "ID": dict1.keys(),
            "Key": dict1.values(),
            "XML Key": [dict2.get(k, "") for k in dict1.keys()],
            "XML Value": [dict3.get(k, "") for k in dict1.keys()],
        }
    )

    # Save the DataFrame to a CSV file
    df.to_csv(output_path, index=False)
    print(f"CSV saved to {output_path}")

In [35]:
combine_and_write_to_csv(cii_path, de_xml_path, "output.csv")

CSV saved to output.csv
